In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [3]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteRRPKR3/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteRRPKR3/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteRRPKR3/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [4]:
encoder = info.features['text'].encoder
encoder

<SubwordTextEncoder vocab_size=8185>

In [5]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64

In [6]:
padded_shapes = ([None], ())

In [7]:
train_data = train_data.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

In [8]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

In [9]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

In [10]:
history = model.fit(train_data, epochs=5, validation_data=test_data, validation_steps=30)

Epoch 1/5
391/391 [==============================] - 110s 256ms/step - loss: 0.6503 - accuracy: 0.6075 - val_loss: 0.5151 - val_accuracy: 0.7750
Epoch 2/5
391/391 [==============================] - 79s 201ms/step - loss: 0.3414 - accuracy: 0.8621 - val_loss: 0.3482 - val_accuracy: 0.8594
Epoch 3/5
391/391 [==============================] - 58s 149ms/step - loss: 0.2533 - accuracy: 0.9062 - val_loss: 0.3320 - val_accuracy: 0.8687
Epoch 4/5
391/391 [==============================] - 55s 140ms/step - loss: 0.2130 - accuracy: 0.9221 - val_loss: 0.3485 - val_accuracy: 0.8740
Epoch 5/5
391/391 [==============================] - 51s 131ms/step - loss: 0.1806 - accuracy: 0.9366 - val_loss: 0.3515 - val_accuracy: 0.8625


In [11]:
def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec))
  vec.extend(zeros)
  return vec

In [12]:
def sample_predict(sentence, pad, model_):
    encoded_sample_pred_text = encoder.encode(sentence)
    if pad:
        encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
    encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
    predictions = model_.predict(tf.expand_dims(encoded_sample_pred_text, 0))

    return predictions

In [13]:
sample_text = ('This movie was awesome. The acting was incredible. Highly recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

1/1 [==============================] - 1s 1s/step


In [14]:
print('probability this is a positive review %.2f' % predictions)

probability this is a positive review 78.65


In [15]:
sample_text = ('This movie was so so. The acting was medicore. Kind of recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

1/1 [==============================] - 0s 92ms/step


In [16]:
print('probability this is a positive review %.2f' % predictions)

probability this is a positive review 42.22


In [17]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                              tf.keras.layers.Dense(64, activation='relu'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1, activation='sigmoid')])

In [18]:
model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(1e-4),
                metrics=['accuracy'])

In [20]:
history = model.fit(train_data, epochs=5, validation_data=test_data,
                    validation_steps=30)

Epoch 1/5
391/391 [==============================] - 110s 258ms/step - loss: 0.6593 - accuracy: 0.5852 - val_loss: 0.5047 - val_accuracy: 0.7708
Epoch 2/5
391/391 [==============================] - 81s 206ms/step - loss: 0.3970 - accuracy: 0.8373 - val_loss: 0.3813 - val_accuracy: 0.8359
Epoch 3/5
391/391 [==============================] - 75s 191ms/step - loss: 0.2890 - accuracy: 0.8953 - val_loss: 0.3330 - val_accuracy: 0.8635
Epoch 4/5
391/391 [==============================] - 72s 184ms/step - loss: 0.2331 - accuracy: 0.9225 - val_loss: 0.3567 - val_accuracy: 0.8510
Epoch 5/5
391/391 [==============================] - 69s 176ms/step - loss: 0.2010 - accuracy: 0.9347 - val_loss: 0.3651 - val_accuracy: 0.8677


In [21]:
sample_text = ('This movie was awesome. The acting was incredible. Highly recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

print('probability this is a positive review %.2f' % predictions)

1/1 [==============================] - 2s 2s/step
probability this is a positive review 58.74


In [22]:
sample_text = ('This movie was so so. The acting was medicore. Kind of recommend')
predictions = sample_predict(sample_text, pad=True, model_=model) * 100

print('probability this is a positive review %.2f' % predictions)

1/1 [==============================] - 0s 34ms/step
probability this is a positive review 14.34
